In [ ]:
#MOD BY CRAB
#更换GPU（需注册） 打开网络
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

#填入视频文件名
filename='3.mp4'
%cd /kaggle/input/tampdata
in_path = os.path.join("/kaggle/input/tampdata", filename)
print(in_path)

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
%cd /kaggle/working
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
# !pip install -r requirements.txt
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
#!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/ESRGAN_SRx4_DF2KOST_official-ff704c30.pth -P experiments/pretrained_models
print("\033[34m done \033[0m")  

In [ ]:
import cv2
video = cv2.VideoCapture(in_path)

fps = video.get(cv2.CAP_PROP_FPS)
print("帧率: {0}".format(fps))
frame_num = video.get(cv2.CAP_PROP_FRAME_COUNT)
print("帧数: {0}".format(frame_num))

In [ ]:
! mkdir -p '/kaggle/working/chunk_in_tmp_video'
! mkdir -p '/kaggle/working/chunk_out_tmp_video'
! mkdir -p '/kaggle/working/frame'
! mkdir -p '/kaggle/working/Real-ESRGAN/frame'
! mkdir -p '/kaggle/working/Real-ESRGAN/results'

#x264 x265参数设置
x=48 #48：264 8bit 410：264 10bit 58：265 8bit 510：265 10bit
crf=16 #[-12-51]
preset='slower' #ultrafast、superfast、veryfast、faster、fast、medium、slow、slower、veryslow、placebo
print("\033[34m done \033[0m")

In [ ]:
#用来测试需要补的chunk是否正确 代码和下一个框一样
tmp=int(frame_num)
print("总帧数：",tmp)

chunk_fps_num=100 
#chunk_fps_num是一个chunk的帧数 建议100  实测超分到900帧会崩溃 建议使用开始chunk和结束chunk多次运行

ran_count = range(0,(tmp), chunk_fps_num)
len_count = len(ran_count) 
print("总chunk数:",len_count)

chunk_end_num_tmp = 20
#chunk_end_num_tmp是终止chunk序号 填0为全部超分
if chunk_end_num_tmp==0:
    chunk_end_num_tmp=len_count
chunk_end_num = len_count - chunk_end_num_tmp 
print("结束chunk：",len_count-chunk_end_num)

chunk_num = 0  
#chunk_num是起始chunk序号 从0开始
print("开始chunk：",chunk_num)

tmp1=tmp +chunk_fps_num-chunk_fps_num*chunk_end_num
ran = range(chunk_num*chunk_fps_num,(tmp1), chunk_fps_num)
#for i in ran :
#    print(i)   
lent = len(ran) 
len1 = lent-1
print("超分chunk数:",len1)

print("超分以下chunk")
for q in range(len1) :
    len1 = lent-1
    q1 = ran[q]
    if chunk_end_num != 0 :
        q2 = ran[q+1]-1
    elif (chunk_end_num == 0) and (q==lent-2-chunk_num):
        q2 = tmp
    elif chunk_end_num == 0 and q!=lent-2-chunk_num:
        q2 = ran[q+1]-1
    print(q1,"--",q2)

In [ ]:
tmp=int(frame_num)
print("总帧数：",tmp)

chunk_fps_num=100
#chunk_fps_num是一个chunk的帧数

ran_count = range(0,(tmp), chunk_fps_num)
len_count = len(ran_count) 
print("总chunk数:",len_count)

chunk_end_num_tmp = 20
#chunk_end_num_tmp是终止chunk序号 填0为全部超分
if chunk_end_num_tmp==0:
    chunk_end_num_tmp=len_count
chunk_end_num = len_count - chunk_end_num_tmp 
print("结束chunk：",len_count-chunk_end_num)

chunk_num = 0  
#chunk_num是起始chunk序号 0开始
print("开始chunk：",chunk_num)

tmp1=tmp +chunk_fps_num-chunk_fps_num*chunk_end_num
ran = range(chunk_num*chunk_fps_num,(tmp1), chunk_fps_num)
#for i in ran :
#    print(i)   
lent = len(ran) 
len1 = lent-1
print("超分chunk数:",len1)

print("超分以下chunk")
for q in range(len1) :
    len1 = lent-1
    q1 = ran[q]
    if chunk_end_num != 0 :
        q2 = ran[q+1]-1
    elif (chunk_end_num == 0) and (q==lent-2-chunk_num):
        q2 = tmp
    elif chunk_end_num == 0 and q!=lent-2-chunk_num:
        q2 = ran[q+1]-1
    print(q1,"--",q2)

for q in range(len1) :
    len1 = lent-1
    q1 = ran[q]
    if chunk_end_num != 0 :
        q2 = ran[q+1]-1
    elif (chunk_end_num == 0) and (q==lent-2-chunk_num):
        q2 = tmp
    elif chunk_end_num == 0 and q!=lent-2-chunk_num:
        q2 = ran[q+1]-1
    print("\033[34m正在运行 ：\033[0m",q1,"--",q2)
     
    tmp_video_inpath = "chunk_in_tmp_"+str(q)+".mp4"
    tmp_video_outpath= "chunk_out_tmp_"+str(q)+".mp4"
    print("chunk 起始帧",q1," 终止帧",q2,"  chunkin文件名：",tmp_video_inpath,"  chunkout文件名：",tmp_video_outpath)
    
    print()
    print("\033[31m切视频\033[0m")                                                             
    !ffmpeg -y -i '/kaggle/input/tampdata/{filename}' -vf "select=between(n\,{q1}\,{q2})"  '/kaggle/working/chunk_in_tmp_video/{tmp_video_inpath}'
    
    print()
    print("\033[31m拆帧\033[0m")
    !ffmpeg -i '/kaggle/working/chunk_in_tmp_video/{tmp_video_inpath}' '/kaggle/working/Real-ESRGAN/frame/%05d.png'
    
    print()
    print("\033[31m超分\033[0m")
    %cd /kaggle/working/Real-ESRGAN 
    # if it is out of memory, try to use the `--tile` option 爆显存调tile 
    !python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input frame --scale 4 --half #--tile 300
    
    print()
    print("\033[31m压制\033[0m")   
    if x==48:
    #264 8bit
        !ffmpeg -r {fps} -i /kaggle/working/Real-ESRGAN/results/%05d_out.png -c:v libx264 -preset:v {preset} -profile:v high -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf {crf} -x264-params "keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709" -pix_fmt yuv420p '/kaggle/working/chunk_out_tmp_video/{tmp_video_outpath}'
    elif x==410:
    #264 10bit
        !ffmpeg -r {fps} -i /kaggle/working/Real-ESRGAN/results/%05d_out.png -c:v libx264 -preset:v {preset} -profile:v high10 -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf {crf} -x264-params "keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709" -pix_fmt yuv420p10 '/kaggle/working/chunk_out_tmp_video/{tmp_video_outpath}'
    elif x==58:
        !ffmpeg -r {fps} -i /kaggle/working/Real-ESRGAN/results/%05d_out.png -c:v libx265 -preset:v {preset} -profile:v main -x265-params crf={crf}:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1 -pix_fmt yuv420p '/kaggle/working/chunk_out_tmp_video/{tmp_video_outpath}'
    elif x==510:    
        !ffmpeg -r {fps} -i /kaggle/working/Real-ESRGAN/results/%05d_out.png -c:v libx265 -preset:v {preset} -profile:v main10 -x265-params crf={crf}:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1 -pix_fmt yuv420p10 '/kaggle/working/chunk_out_tmp_video/{tmp_video_outpath}' 
    
    print()
    print("\033[31m清理\033[0m")
    ! rm -rf '/kaggle/working/frame'
    ! mkdir -p '/kaggle/working/frame'
    ! rm -rf '/kaggle/working/Real-ESRGAN/frame'
    ! mkdir -p '/kaggle/working/Real-ESRGAN/frame'
    ! rm -rf '/kaggle/working/Real-ESRGAN/results'
    ! mkdir -p '/kaggle/working/Real-ESRGAN/results'
    print("清理完成")
    print()
    print()

总帧数： 7949
总chunk数: 80
结束chunk： 20
开始chunk： 0
超分chunk数: 20
超分以下chunk
0 -- 99
100 -- 199
200 -- 299
300 -- 399
400 -- 499
500 -- 599
600 -- 699
700 -- 799
800 -- 899
900 -- 999
1000 -- 1099
1100 -- 1199
1200 -- 1299
1300 -- 1399
1400 -- 1499
1500 -- 1599
1600 -- 1699
1700 -- 1799
1800 -- 1899
1900 -- 1999
正在运行 ： 0 -- 99
chunk 起始帧 0  终止帧 99   chunkin文件名： chunk_in_tmp_0.mp4   chunkout文件名： chunk_out_tmp_0.mp4

切视频
ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feed

In [ ]:
#%cd /kaggle/working/Real-ESRGAN
#!ls
#kaggle的git有问题 报错inference_realesrgan.py找不到请直接重启

In [ ]:
#视频合并
%cd /kaggle/working/chunk_out_tmp_video
for q in range(len1) :
    tmp_video_outpath1= "file 'chunk_out_tmp_"+str(q)+".mkv'"
    f = open("merge.txt",'a')
    f.write(tmp_video_outpath1)
    f.write('\n')
    f.close()
    #print(1)
    
%cd /kaggle/working/chunk_out_tmp_video
!ffmpeg -y -f concat -i merge.txt -c copy /kaggle/working/merge.mkv
!rm merge.txt


In [ ]:
#测试帧数
video_tmp_path = '/kaggle/working/merge.mkv'
video_tmp = cv2.VideoCapture(video_tmp_path)

fps_out_tmp = video_tmp.get(cv2.CAP_PROP_FPS)
print("帧率: {0}".format(fps_out_tmp))
frame_out_tmp = video_tmp.get(cv2.CAP_PROP_FRAME_COUNT)
print("帧数: {0}".format(frame_out_tmp))
